In [54]:
import os 
os.chdir("/Users/tonyzhang/Desktop/Chicken-Disease-Classification/chicken-disease-classification/")
print(os.getcwd())

/Users/tonyzhang/Desktop/Chicken-Disease-Classification/chicken-disease-classification


In [55]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str
    local_data_file: Path 
    unzip_dir: Path 

In [56]:
from cnnClassifier.constants import * 
from cnnClassifier.utils.common import read_yaml, create_directories

In [57]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,
                source_URL= config.source_URL,
                local_data_file = config.local_data_file,
                unzip_dir= config.unzip_dir
          )

          return data_ingestion_config

In [58]:
import os 
import zipfile
import urllib.request as request 
from cnnClassifier import logger 
from cnnClassifier.utils.common import get_size

In [59]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded {filename} with the following info: {headers}")
        else:
            logger.info(f"File already exists of size {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [60]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-08-18 00:36:26,397: INFO: common: Successully read the yaml file at config/config.yaml]
[2024-08-18 00:36:26,400: INFO: common: Successully read the yaml file at params.yaml]
[2024-08-18 00:36:26,402: INFO: common: Created directory at artifacts]
[2024-08-18 00:36:26,404: INFO: common: Created directory at artifacts/data_ingestion]
[2024-08-18 00:36:26,405: INFO: 3167468587: File already exists of size ~11344.64 KB]
